<a href="https://colab.research.google.com/github/LizLian/from_scratch_2025/blob/main/%5BG%5Dgpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-11 15:42:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com... failed: Name or service not known.
wget: unable to resolve host address 'raw.githubusercontent.com'


In [ ]:
from google3.pyglib import gfile

In [ ]:
# read it in to inspect it
with gfile.Open('/cns/wf-d/home/lizliang/dataset/from_scratch/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: takes a strign, outputs a list of integers
decode = lambda l: "".join([itos[i] for i in l]) # decoder: takes a list of integers, outputs a string

print(encode("hii there"))
print(decode([46, 47, 47, 1, 58, 46, 43, 56, 43]))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
# encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
# split up the data into train and validation sets
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [ ]:
torch.manual_seed(1337)
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# -------------------------------

def get_batch(split):
  data = train_data if split=="train" else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print("targets:")
print(yb.shape)

cuda
inputs:
torch.Size([64, 256])
targets:
torch.Size([64, 256])


In [ ]:
def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
tril = torch.tril(torch.ones(block_size, block_size))

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class Head(nn.Module):
  """ one head of self-attention """

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    # input (batch, timestep, channels)
    # output (batch, timestep, head size)
    B,T,C = x.shape
    k = self.key(x) # (B,T,hs)
    q = self.query(x) # (B,T,hs)
    # compute attention scores
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,T)
    wei = wei.masked_fill(self.tril[:T, :T]==0, float("-inf"))
    wei = F.softmax(wei, dim=-1) #(B,T,T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values
    v = self.value(x) #(B,T,C)
    out = wei @ v
    return out


class MultiHeadAttention(nn.Module):
  """ multiple heads of self-attention in parallel """

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(head_size * num_heads, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out


In [ ]:
# @title FeedForward Network
class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

In [ ]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [ ]:
# @title Bigram language model pytorch implementation

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    # self.blocks = nn.Sequential(
    #     Block(n_embd, n_head=4),
    #     Block(n_embd, n_head=4),
    #     Block(n_embd, n_head=4),
    # )
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb # (B, T, C)
    x = self.blocks(x) # B, T, C
    x = self.ln_f(x) # B, T, C
    logits = self.lm_head(x) # (B, T, vocab_size)

    loss = None
    if targets is not None:
      B, T, C = logits.shape
      logits = logits.view(B, C, T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, "M parameters")


10.788929 M parameters


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for iter in range(max_iters):
  # every once in a while evaluate the loss on train and eval sets
  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}")

  # sample a batch of data
  xb, yb = get_batch("train")

  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.3314, val loss 4.3312
step 500: train loss 3.1421, val loss 3.1818
step 1000: train loss 1.6729, val loss 1.7189
step 1500: train loss 1.2455, val loss 1.3043
step 2000: train loss 1.1617, val loss 1.2191
step 2500: train loss 1.1307, val loss 1.1807
step 3000: train loss 1.1160, val loss 1.1657
step 3500: train loss 1.1060, val loss 1.1571
step 4000: train loss 1.0996, val loss 1.1527
step 4500: train loss 1.0952, val loss 1.1501
step 4999: train loss 1.0920, val loss 1.1427


In [ ]:
# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


DGoZuSPEJprULm
zFVNz
3;-3S:oXicDW!N?U?
mPRBeM$;nuaUzWpzmAP' 
?V 
kA&yg',Ysql
zd$LX'WwIZ?PAR'v3hkDpf3fKfH!ZW:CVwZkE?cD:mWHQSylC,b-dVYx'C::hJ-MGOfe!pjU?UyijKTTSYTLw
DVBbNM, Yc?rTe-G--ZbFCX'QY!MDAzMJV'pzZhf&MVS:l:OzoT;ik $AhH!CDb zEFgzP-WzBiezWWi&GIOWq;QzUzXBbaBzN?P,V&3VZjANjiqDBGg?H?ZCRAlUd!lEU&TmS$mOaMzeHl$t&$LqV: ibxGuE3:XQ-,':ZFzju!:b$-!o;PZVfMe;S,kCIGRNgrK Nk;KFlC3DKLVCcZ,NST :A'yZpEzfJEUzN.GKiEByI.:BhpNXE.,QIJ D--$-AuCwp Bd'cd?3ZVK$QG.L3-ofkfCWGq&ELQ 'BXEZf'hQF!ewnmD
b?RM:ruHXDQU&pPlH.PQtbQIT
